In [3]:
# Read data
JD_data <- read.csv("JD.csv")

# Calculate the daily logarithmic rate of return
JD_data$Return <- c(NA, diff(log(JD_data$Adj.Close)))

# Remove rows containing NA
JD_data <- na.omit(JD_data)

# Output the calculated yield
print(JD_data$Return)

  [1]  0.0038923462 -0.0436730013 -0.0050257136 -0.0208492401  0.0000000000
  [6] -0.0410057449  0.0073745025 -0.0277439320  0.0139681787  0.0005136364
 [11] -0.0041162908 -0.0106259906  0.0041601723  0.0235907863 -0.0015216082
 [16] -0.0207726915 -0.0167236454  0.0149080567 -0.0065112875 -0.0060280355
 [21] -0.0013152441 -0.0121824333 -0.0174711607 -0.0378551352  0.0042153218
 [26] -0.0098633742  0.0445871172 -0.0076128430 -0.0118052164 -0.0294262233
 [31]  0.0197134232  0.0116441016  0.0002756511 -0.0033122581 -0.0074926395
 [36]  0.0165749094 -0.0137933221  0.0005554013 -0.0064057870 -0.0203224886
 [41] -0.0348164268 -0.0456065142 -0.0121251913  0.0077894770  0.0251315701
 [46] -0.0072904940 -0.0141236308 -0.0298155152  0.0003185539  0.0396492715
 [51] -0.0157337201 -0.0065512322 -0.0301850076  0.0096308611 -0.0616035034
 [56] -0.1124456611  0.0354846199 -0.0110701866 -0.0157071286  0.0208884503
 [61]  0.0197374466  0.0046940329 -0.0236934352 -0.0522450741  0.0007770008
 [66]  0.044

In [4]:
# Define the PDF of the Generalized t-distribution
dt_G <- function(x, params) {
  location <- params[1]
  scale <- params[2]
  shape <- params[3]
  return(dt((x - location) / scale, df = shape) / scale)
}

# Define the CDF of the Generalized t-distribution
pt_G <- function(q, location, scale, shape) {
  return(pt((q - location) / scale, df = shape))
}

In [5]:
# Define the negative log-likelihood function for Maximum Likelihood Estimation (MLE)
neg_log_likelihood <- function(params, data) {
  log_density <- sapply(data, function(x) log(dt_G(x, params)))
  return(-sum(log_density))
}

# Prepare return data and initial parameter estimates
returns_data <- JD_data$Return
initial_params <- c(location = mean(returns_data), scale = sd(returns_data), shape = 5)

# Perform MLE to estimate the parameters of the Generalized t-distribution
mle_result <- optim(par = initial_params, fn = neg_log_likelihood, data = returns_data,
                    method = "L-BFGS-B", lower = c(-Inf, 0.001, 2), upper = c(Inf, Inf, Inf))
mle_params <- mle_result$par

# Print the parameters estimated by MLE
cat("MLE Results:\n")
print(mle_params)

MLE Results:
    location        scale        shape 
-0.001561329  0.025491936  6.711448675 


In [10]:
# Load the fitdistrplus package for additional fitting methods
library(fitdistrplus)

# Define the PDF and CDF for use with fitdistrplus
ddt_G_fitdistr <- function(x, location, scale, shape) {
  return(dt((x - location) / scale, df = shape) / scale)
}
pdt_G_fitdistr <- function(q, location, scale, shape) {
  return(pt((q - location) / scale, df = shape))
}

# Fit the distribution using fitdistrplus with initial parameter estimates
fit_result <- fitdist(JD_data$Return, "dt_G_fitdistr", start = list(location = mean(returns_data), scale = sd(returns_data), shape = 5))

# Print the parameters estimated by fitdistrplus
cat("\nfitdistrplus Results:\n")
print(fit_result$estimate)


fitdistrplus Results:
    location        scale        shape 
-0.001564296  0.025456360  6.708586251 
